In [4]:
!pip3 install "modelscope==1.7.2rc0" -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: modelscope
    Found existing installation: modelscope 1.7.0
    Uninstalling modelscope-1.7.0:
      Successfully uninstalled modelscope-1.7.0


In [1]:
import modelscope
modelscope.__version__

2023-07-21 02:18:46,345 - modelscope - INFO - PyTorch version 2.0.1+cu117 Found.
2023-07-21 02:18:46,350 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-07-21 02:18:46,379 - modelscope - INFO - Loading done! Current index file version is 1.7.2rc0, with md5 fe01e78b8984d1ed9e454203cb44a964 and a total number of 863 components indexed


'1.7.2rc0'

In [2]:
# read questions
import json

ques_file = '/mnt/workspace/llm-eval/data/question.jsonl'

ques_list = []
with open(ques_file, 'r') as f:
    for line in f:
        line = line.strip()
        ques_list.append(json.loads(line))

print(len(ques_list))
print(next(iter(ques_list)))

80
{'question_id': 1, 'text': 'How can I improve my time management skills?', 'category': 'generic'}


In [4]:
import json
import time

def get_answers(model_name, model, tokenizer, ques_list, out_file):
    
    with open(out_file, 'a') as f:
        for ques in ques_list:
            print('\n>>>inputs: ', ques)
            start = time.time()
            
            # do infer
            inputs = tokenizer(ques['text'], return_tensors="pt")
            generate_ids = model.generate(inputs.input_ids, max_length=30)
            result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            
            
            print('***Time cost: ', time.time() - start)

            res = {'model_id': model_name, 'metadata': {}, 'answer': result}    # todo !!!  result
            ques.update(res)
            print('\n*** res: ', ques)
            
            f.write(json.dumps(ques, ensure_ascii=False) + '\n')

    print('** Dump data finished !\n')


In [5]:
import torch
from modelscope import snapshot_download, Model
from modelscope.models.nlp.llama2 import Llama2Tokenizer

model_dir = snapshot_download("modelscope/Llama-2-7b-ms", revision='v1.0.1', 
                              ignore_file_pattern = [r'\w+\.safetensors'])
model = Model.from_pretrained(model_dir, device_map='auto', torch_dtype=torch.float16)
tokenizer = Llama2Tokenizer.from_pretrained(model_dir)


2023-07-21 02:18:56,737 - modelscope - INFO - Use user-specified model revision: v1.0.1
2023-07-21 02:18:57,155 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/modelscope/Llama-2-7b-ms


[2023-07-21 02:18:57,293] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-21 02:18:57.771922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'Llama2Tokenizer'.


In [6]:
# ONLY FOR TEST 

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Hey, are you conscious? Can you talk to me?
I'm not sure if you're conscious or not.
I'


In [7]:
model_name = 'llama2-7b'
out_file = '/mnt/workspace/llm-eval/data/answer_llama2-7b.jsonl'

get_answers(model_name, model, tokenizer, ques_list, out_file)



>>>inputs:  {'question_id': 1, 'text': 'How can I improve my time management skills?', 'category': 'generic'}
***Time cost:  0.7945966720581055

*** res:  {'question_id': 1, 'text': 'How can I improve my time management skills?', 'category': 'generic', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'How can I improve my time management skills?\nHow can I improve my time management skills? 10 Tips for Improving Your'}

>>>inputs:  {'question_id': 2, 'text': 'What are the most effective ways to deal with stress?', 'category': 'generic'}
***Time cost:  0.7123160362243652

*** res:  {'question_id': 2, 'text': 'What are the most effective ways to deal with stress?', 'category': 'generic', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'What are the most effective ways to deal with stress?\nWhat are the most effective ways to deal with stress? What are the most effective ways'}

>>>inputs:  {'question_id': 3, 'text': 'What are the main differences between Python and JavaScript progr

Input length of input_ids is 31, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


***Time cost:  0.5672097206115723

*** res:  {'question_id': 10, 'text': 'What are the most effective strategies for conflict resolution in the workplace?', 'category': 'generic', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'What are the most effective strategies for conflict resolution in the workplace?\nWhat are the most effective strategies for conflict resolution in the work'}

>>>inputs:  {'question_id': 11, 'text': 'What are some potential implications of using a single-use plastic bottle versus a reusable bottle on both the environment and human health?', 'category': 'knowledge'}
***Time cost:  0.044432640075683594

*** res:  {'question_id': 11, 'text': 'What are some potential implications of using a single-use plastic bottle versus a reusable bottle on both the environment and human health?', 'category': 'knowledge', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'What are some potential implications of using a single-use plastic bottle versus a reusable bottle on b

Input length of input_ids is 31, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


***Time cost:  0.3225388526916504

*** res:  {'question_id': 15, 'text': 'Describe a scenario where artificial intelligence could be used to improve the quality and efficiency of healthcare delivery.', 'category': 'knowledge', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Describe a scenario where artificial intelligence could be used to improve the quality and efficiency of healthcare delivery.\nDescribe a scenario where artificial intelligence'}

>>>inputs:  {'question_id': 16, 'text': 'Explain the process of gene editing using CRISPR-Cas9 technology, and discuss its potential applications and ethical implications.', 'category': 'knowledge'}
***Time cost:  0.04123520851135254

*** res:  {'question_id': 16, 'text': 'Explain the process of gene editing using CRISPR-Cas9 technology, and discuss its potential applications and ethical implications.', 'category': 'knowledge', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Explain the process of gene editing using CRISPR-Cas9 tec

...

***Time cost:  0.2520482540130615

*** res:  {'question_id': 69, 'text': 'Solve for x in the equation 3x + 10 = 5(x - 2).', 'category': 'math', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Solve for x in the equation 3x + 10 = 5(x - 2).\nSolve for x in'}

>>>inputs:  {'question_id': 70, 'text': 'If the endpoints of a line segment are (2, -2) and (10, 4), what is the length of the segment?', 'category': 'math'}
***Time cost:  0.04380679130554199

*** res:  {'question_id': 70, 'text': 'If the endpoints of a line segment are (2, -2) and (10, 4), what is the length of the segment?', 'category': 'math', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'If the endpoints of a line segment are (2, -2) and (10, 4), what is the length of the segment?\n'}

>>>inputs:  {'question_id': 71, 'text': 'Can you help me write a formal email to a potential business partner proposing a joint venture?', 'category': 'writing'}


Input length of input_ids is 32, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


***Time cost:  0.37207937240600586

*** res:  {'question_id': 71, 'text': 'Can you help me write a formal email to a potential business partner proposing a joint venture?', 'category': 'writing', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Can you help me write a formal email to a potential business partner proposing a joint venture?\nI am a businessman from the United'}

>>>inputs:  {'question_id': 72, 'text': 'Can you help me write a resignation letter to my current employer, while leaving on good terms and expressing gratitude for the opportunities provided?', 'category': 'writing'}
***Time cost:  0.04265594482421875

*** res:  {'question_id': 72, 'text': 'Can you help me write a resignation letter to my current employer, while leaving on good terms and expressing gratitude for the opportunities provided?', 'category': 'writing', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Can you help me write a resignation letter to my current employer, while leaving on good terms 

Input length of input_ids is 30, but `max_length` is set to 30. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


***Time cost:  0.40613722801208496

*** res:  {'question_id': 74, 'text': 'Write a compelling product launch announcement email to inform our customers of our new software solution.', 'category': 'writing', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Write a compelling product launch announcement email to inform our customers of our new software solution.\nWrite a compelling product launch announcement email'}

>>>inputs:  {'question_id': 75, 'text': 'Draft an apology email to a customer who experienced a delay in their order, and provide reassurance that the issue has been resolved.', 'category': 'writing'}
***Time cost:  0.041651010513305664

*** res:  {'question_id': 75, 'text': 'Draft an apology email to a customer who experienced a delay in their order, and provide reassurance that the issue has been resolved.', 'category': 'writing', 'model_id': 'llama2-7b', 'metadata': {}, 'answer': 'Draft an apology email to a customer who experienced a delay in their order, and provide